# RAG Troubleshooting & Tips

Шпаргалка по частым ошибкам, багам и полезным приёмам для RAG-ноутбуков.

Можно положить в репозиторий как `RAG_Troubleshooting.ipynb` или `docs/RAG_troubleshooting.md`.


## 0. Структура репозитория (рекомендация)

Пример структуры репы с RAG-ноутбуками:

```text
rag-project/
  ├─ README.md
  ├─ RAG_Baseline_AutoRAG.ipynb              # основной пайплайн (autorag_style_rag_pipeline)
  ├─ RAG_Simple_Playground.ipynb             # простой playground (simple_rag_playground)
  ├─ RAG_Experiments_Eval_Lab.ipynb          # лаборатория экспериментов (rag_experiments_eval_lab)
  ├─ RAG_Troubleshooting.ipynb               # этот файл
  ├─ configs/
  │    ├─ rag_config_example.yaml            # по желанию — конфиги
  └─ data/
       ├─ docs.csv / docs/                   # локальные данные
       ├─ qa.csv
       └─ sample_submission.csv
```

Идея: один baseline, один playground, одна «лаборатория» и один файл со шпаргалкой.


## 1. Быстрая проверка окружения

In [ ]:
import torch, transformers, datasets, platform, sys
print('Python :', sys.version)
print('OS     :', platform.platform())
print('Torch  :', torch.__version__)
print('GPU    :', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU only')
print('HF     :', transformers.__version__)
print('datasets:', datasets.__version__)


Если что-то падает, сначала убедись, что:

- `torch` видит GPU;
- версии `transformers` и `sentence-transformers` совместимы (в кластерных средах это может быть жёстко зафиксировано);
- ты не на Python 3.12 с очень старыми библиотеками (или наоборот).


## 2. Проблемы с загрузкой моделей/токенайзеров HF

Частые ошибки:

- `OSError: Can't load tokenizer for '...'`;
- `We couldn't connect to 'https://huggingface.co' ...` (нет интернета);
- зависание при скачивании модели.

### Решения

1. **Используй локальный путь**, а не имя из HF Hub.

```python
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH = '/kaggle/input/qwen2.5/transformers/1.5b-instruct/1'

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, local_files_only=True)
```

2. Для `SentenceTransformer`:

```python
from sentence_transformers import SentenceTransformer

EMBED_MODEL_PATH = '/kaggle/input/intfloat-multilingual-e5-large'
model = SentenceTransformer(EMBED_MODEL_PATH, device='cuda')  # без похода в интернет
```

3. Если вообще нет токенайзера:

- проверь, что в папке есть `tokenizer.json` или `tokenizer_config.json`;
- если модель quantized / кастомная, иногда нужно ставить правильную версию `transformers`.


## 3. CUDA out of memory (OOM)

Симптом: `RuntimeError: CUDA out of memory ...`

### Как лечить

1. **Уменьшить batch size** при эмбеддинге:

```python
vectors = embedding_model.encode(
    texts,
    batch_size=16,  # было 64
    normalize_embeddings=True,
    show_progress_bar=True,
)
```

2. **Уменьшить размер чанка / число чанков**:

- `CHUNK_SIZE` поменьше,
- `CHUNK_OVERLAP` поменьше
(меньше текста → меньше RAM и VRAM).

3. **Уменьшить TOP-K** в retrieval:

- `TOP_K_RETRIEVAL`, `TOP_K_FINAL` сделали меньше;
- меньше контекста — меньше токенов у LLM.

4. **Использовать float16** для генерации (в ноутбуках уже стоит):

```python
generation_pipeline = pipeline(
    'text-generation',
    model=GEN_MODEL_PATH,
    device=0,
    torch_dtype=torch.float16,
)
```

5. **Чистить GPU-память между большими шагами** (если прям прижало):

```python
import torch, gc
gc.collect()
torch.cuda.empty_cache()
```


## 4. Expected all tensors to be on the same device

Ошибка вида:

`RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!`

### Причины

- модель на GPU, а входной тензор на CPU (или наоборот);
- несколько моделей с разными `device`.

### Что делать

1. Убедись, что везде используешь один и тот же `device`:

```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embedding_model = SentenceTransformer(EMBED_MODEL_ID, device=str(device))
generation_pipeline = pipeline(
    'text-generation',
    model=GEN_MODEL_PATH,
    device=0 if device.type == 'cuda' else -1,
)
```

2. Если где-то вручную создаёшь тензоры, явно переносить на `device`:

```python
q_vec = torch.tensor(q_vec).to(device)
```


## 5. Ошибки с `pad_token_id` / `eos_token_id`

Часто у LLM (особенно Qwen/LLaMA-подобных) нет `pad_token_id` по умолчанию.

Ошибка:

- `ValueError: You have to specify either pad_token_id or eos_token_id`;
- или предупреждения от `transformers`.

### Быстрый фикс

Если используешь `pipeline`, можно забрать `eos_token_id` из токенайзера и пробросить как `pad_token_id`:

```python
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_PATH = GEN_MODEL_PATH
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

generation_pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)
```

В наших ноутбуках обычно хватает передачи `pad_token_id=generation_pipeline.tokenizer.eos_token_id` в `pipeline(...).`


## 6. `datasets` + отсутствие интернета

Если среда без интернета (кластер, финал соревнования), то `load_dataset(...)` может упасть при первой загрузке.

### Стратегия

1. **Заранее выгрузи датасет в CSV/Parquet** и клади в `/kaggle/input` / локальную папку.

```python
from datasets import load_dataset

ds = load_dataset('your/dataset', split='train')
df = ds.to_pandas()
df.to_csv('docs.csv', index=False)
```

2. В боевых ноутбуках вместо `load_dataset` используй `pd.read_csv`.

3. Если очень надо `datasets`, можно передать `cache_dir` и надеяться, что кэш уже есть в среде:

```python
ds = load_dataset('your/dataset', split='train', cache_dir='/kaggle/input/hf-cache')
```


## 7. Ошибки с колонками (`KeyError`, `ValueError`)

Типичные кейсы:

- `KeyError: 'question'` — в QA-файле колонка называется иначе (`text`, `query` и т.д.);
- `ValueError: Колонка ... не найдена` — в наших helper-функциях жёстко зашито имя.

### Что делать

1. Сразу после чтения данных **печатай первые строки** и список колонок:

```python
print(qa_df.columns)
qa_df.head()
```

2. Приведи всё к стандартным именам в одном месте:

```python
qa_df = qa_df.rename(columns={
    'query_text': 'question',
    'id_col': 'id',
})
```

3. В конфиге ноутбука собери все важные имена колонок:

```python
QA_ID_COL = 'id'
QA_QUESTION_COL = 'question'
QA_GT_ANSWER_COL = 'answer'
```


## 8. Как дебажить качество ретрива (до LLM)

Чтобы понять, что RAG тупит из‑за плохого retriever, полезно смотреть, **какие чанки достаются**, без LLM.

Пример helper-функции (можно переиспользовать в любых ноутбуках):


In [ ]:
def inspect_retrieval(query: str, mode: str = 'hybrid', top_k: int = 5):
    if mode == 'dense':
        retrieved = dense_search(query, top_k=top_k)
    elif mode == 'bm25':
        retrieved = bm25_search(query, top_k=top_k)
    else:
        # для hybrid_search в некоторых ноутах другая сигнатура — подстрой
        retrieved = hybrid_search(query, top_k_final=top_k, top_k_dense=top_k, top_k_bm25=top_k, alpha=0.5)

    print('QUESTION:')
    print(query)
    print('\nRETRIEVED CHUNKS:')
    for i, row in retrieved.iterrows():
        print('=' * 80)
        print(f"rank={i} | doc_id={row['doc_id']} | chunk_id={row['chunk_id']}")
        for col in ['score_dense', 'score_bm25', 'score_hybrid']:
            if col in row and not pd.isna(row[col]):
                print(f"{col} = {row[col]:.4f}")
        print('-' * 40)
        print(row['text'][:800])
        print('\n')

# пример:
# inspect_retrieval('What is the main idea of the document?', mode='hybrid', top_k=5)


## 9. Повторяемость (seed)

Чтобы результаты экспериментов были воспроизводимыми:

```python
import random, numpy as np, torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
```

В наших ноутбуках это уже есть, но при копипасте отдельных кусков кода не забудь перенести блок с SEED.


## 10. Небольшие performance‑советы

- Эмбеддинги:
  - Один раз посчитай эмбеддинги для всех чанков и **сохрани в `.npy`/`.parquet`**, чтобы не пересчитывать при каждом эксперименте.

  ```python
  np.save('dense_matrix.npy', dense_matrix)
  # а потом
  dense_matrix = np.load('dense_matrix.npy')
  ```

- Retrieval:
  - експериментируй с `TOP_K_RETRIEVAL` / `TOP_K_FINAL`: часто 8–10 чанков достаточно;
  - не тащи 50 чанков в LLM, если лимит токенов маленький.

- LLM:
  - держи промпт максимально коротким;
  - выключи `do_sample=True` для более стабильных результатов метрик (для отбора лучшей конфигурации), а потом включай stochastics для финального сабмита.

```python
out = generation_pipeline(
    prompt,
    max_new_tokens=MAX_NEW_TOKENS,
    do_sample=False,  # для оффлайн-оценки
)
```
